In [3]:
import pandas as pd
import numpy as np
import random as random
import pyarrow as pa 
import pyarrow.parquet as pq
from fastavro import writer, reader, parse_schema
import collections
import ta
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os
import gym
import pandas as pd
import numpy as np
from numpy import inf
from gym import spaces
from sklearn import preprocessing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from empyrical import sortino_ratio, calmar_ratio, omega_ratio
from termcolor import colored

In [23]:
window_size=90
current_step = 1
scaler = preprocessing.MinMaxScaler()

In [6]:
df = pq.read_table('../data/raw/check.parquet').to_pandas()

In [8]:
df.rename(
            columns={
                "timestamp_ms": "window_end"
            }, 
            inplace=True
        )
df.set_index(['window_end'], inplace=True)
df.sort_index(inplace=True)

In [44]:
df = df[:1000]
account_history = np.random.rand(100,8)

In [21]:
features = df[current_step: current_step + window_size]

In [64]:
np.diff(np.log(np.ones((90,8))), axis=1)

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
 

In [53]:
acc_feat = pd.DataFrame(account_history[-window_size:], index=features.index, columns=[
            'quote_held',
            'base_held',
            'base_debt',
            'quote_debt',
            'cost',
            'sales',
            'base_sold',
            'base_bought'
])

In [54]:
x = features.merge(
    acc_feat, 
    how='outer', 
    left_index=True, 
    right_index=True
)

In [55]:
len(x.values.flatten())

5580

In [ ]:
scaled_history = scaler.fit_transform(self.account_history[:self.forcasting].astype('float32'))

In [13]:
features = df[df.columns.difference(['index', 'window_end'])]

In [4]:
def _next_observation(self):
    scaler = preprocessing.MinMaxScaler()

    

    scaled = features[:self.current_step + self.forecast_len + 1].values
    scaled[abs(scaled) == inf] = 0
    scaled = scaler.fit_transform(scaled.astype('float32'))
    scaled = pd.DataFrame(scaled, columns=features.columns)

    obs = scaled.values[-1]

    past_df = self.stationary_df['close'][:self.current_step + self.forecast_len + 1]
    forecast_model = SARIMAX(past_df.values, enforce_stationarity=False, simple_differencing=True)
    model_fit = forecast_model.fit(method='bfgs', disp=False)
    forecast = model_fit.get_forecast(
        steps=self.forecast_len, 
        alpha=(1 - self.confidence_interval)
    )

    obs = np.insert(obs, len(obs), forecast.predicted_mean, axis=0)
    obs = np.insert(obs, len(obs), forecast.conf_int().flatten(), axis=0)

    scaled_history = scaler.fit_transform(self.account_history.astype('float32'))

    obs = np.insert(obs, len(obs), scaled_history[:, -1], axis=0)

    obs = np.reshape(obs.astype('float16'), self.obs_shape)
    obs[np.bitwise_not(np.isfinite(obs))] = 0

    return obs
